
Final_Bert == 768 Features Extracted

In [1]:
!pip install transformers
import transformers as ppb 
import re 
from bs4 import BeautifulSoup 
import numpy as np
import torch
import pandas as pd 
import nltk
nltk.download('stopwords')
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.0 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
import os 
import numpy as np
import pandas as pd
from sklearn import preprocessing 
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedShuffleSplit

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]') 
STOPWORDS =nltk.corpus.stopwords.words('english') 

In [5]:
path = "/content/drive/MyDrive/newpro/threeclass_problem/data.csv"
df = pd.read_csv(path)
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [6]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [7]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [8]:
#define bins
bins = [0,2,3,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names) 

In [9]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [10]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [11]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [12]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel,ppb.BertTokenizer, 'bert-base-cased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t), df.iloc[:,0])) 

In [14]:
train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids,train_tokens))

In [15]:
b=np.zeros((np.shape(df)[0]))
for i in range(0,np.shape(df)[0]):
  b[i]=np.shape(train_tokens_ids[i])[0]

In [16]:
pad=np.zeros((np.shape(df)[0],int(np.max(b))))
for i in range(0,np.shape(df)[0]):
  a=train_tokens_ids[i]
  for j in range(0,np.shape(a)[0]):
    pad[i,j]=a[j]

In [17]:
pad1=pad[:,0:150]       #pad1=pad[:,0:300]
input_ids = torch.tensor(np.array(pad1))        
with torch.no_grad():
  last_hidden_states = model(input_ids.long()) 
features = last_hidden_states[0][:,0,:].numpy()

In [18]:
features

array([[ 0.2515822 ,  0.06544148,  0.03632676, ...,  0.43549427,
        -0.04873884,  0.02371806],
       [ 0.30734295,  0.04622684,  0.07097933, ...,  0.33745626,
         0.15762301,  0.42273486],
       [ 0.3303049 ,  0.21474215,  0.51723254, ..., -0.03585206,
        -0.29864782,  0.32629228],
       ...,
       [ 0.17179213, -0.04870527, -0.00776932, ...,  0.08644193,
         0.2850138 ,  0.40955907],
       [ 0.28869393, -0.01770299, -0.14143702, ...,  0.10606311,
         0.26678324,  0.417692  ],
       [ 0.19270174, -0.14026037, -0.39399457, ..., -0.12173665,
         0.10634034,  0.5625396 ]], dtype=float32)

In [19]:
len(features)

1744

In [20]:
DF = pd.DataFrame(features)
from google.colab import files
DF.to_csv('bert.csv') 
files.download('bert.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
path = "/content/drive/MyDrive/newpro/threeclass_problem/bert.csv" 
x = pd.read_csv(path)
x.head() 

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,758,759,760,761,762,763,764,765,766,767
0,0,0.251582,0.065441,0.036327,-0.683447,-0.864267,0.043688,-0.182814,-0.056845,0.180367,...,0.731047,0.057088,0.038299,0.289821,-0.283759,0.252248,0.018195,0.435494,-0.048739,0.023718
1,1,0.307343,0.046227,0.070979,-0.689943,-0.421657,-0.045189,0.146107,-0.190608,0.162453,...,0.758684,0.373472,0.056404,0.079221,-0.368649,0.135729,-0.081308,0.337456,0.157623,0.422735
2,2,0.330305,0.214742,0.517233,-0.529509,-0.374165,-0.236629,0.780357,-0.248850,0.348099,...,0.462937,0.052816,-0.340460,0.361714,-0.405706,0.020661,-0.041318,-0.035852,-0.298648,0.326292
3,3,0.266967,0.254776,0.249469,-0.518748,-0.460016,-0.099519,0.359582,0.087548,0.356397,...,0.631750,0.190572,0.242963,0.207410,-0.466586,-0.032446,0.130334,0.219458,0.072517,0.343598
4,4,0.221523,0.013097,-0.097723,-1.067212,-0.829449,0.143097,-0.051330,-0.085799,0.035921,...,0.565033,0.261153,-0.027681,0.528556,-0.489401,0.421287,-0.566908,0.184935,0.217472,0.156604


In [22]:
x.shape

(1744, 769)

In [23]:
x.drop(x.columns[[0]], axis=1, inplace=True)

In [24]:
x.shape 

(1744, 768)

In [25]:
x.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.251582,0.065441,0.036327,-0.683447,-0.864267,0.043688,-0.182814,-0.056845,0.180367,-0.370679,...,0.731047,0.057088,0.038299,0.289821,-0.283759,0.252248,0.018195,0.435494,-0.048739,0.023718
1,0.307343,0.046227,0.070979,-0.689943,-0.421657,-0.045189,0.146107,-0.190608,0.162453,-0.529358,...,0.758684,0.373472,0.056404,0.079221,-0.368649,0.135729,-0.081308,0.337456,0.157623,0.422735
2,0.330305,0.214742,0.517233,-0.529509,-0.374165,-0.236629,0.780357,-0.248850,0.348099,-0.530329,...,0.462937,0.052816,-0.340460,0.361714,-0.405706,0.020661,-0.041318,-0.035852,-0.298648,0.326292
3,0.266967,0.254776,0.249469,-0.518748,-0.460016,-0.099519,0.359582,0.087548,0.356397,-0.599168,...,0.631750,0.190572,0.242963,0.207410,-0.466586,-0.032446,0.130334,0.219458,0.072517,0.343598
4,0.221523,0.013097,-0.097723,-1.067212,-0.829449,0.143097,-0.051330,-0.085799,0.035921,0.215113,...,0.565033,0.261153,-0.027681,0.528556,-0.489401,0.421287,-0.566908,0.184935,0.217472,0.156604


In [26]:
y=df['Level ']

In [27]:
import warnings

warnings.filterwarnings("ignore")

In [28]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 768), (524, 768))

In [29]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

3    48.196721
1    29.590164
2    22.213115
Name: Level , dtype: float64


3    46.946565
1    29.961832
2    23.091603
Name: Level , dtype: float64


Perceptron

In [30]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150) 
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [31]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.6221826515482237


In [32]:
import warnings
warnings.filterwarnings("ignore")

**Decision Tree**

In [33]:
clf_dt = DecisionTreeClassifier(max_depth=5)

In [34]:
clf_dt.fit(X_train,y_train) 

DecisionTreeClassifier(max_depth=5)

In [35]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [36]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))


In [37]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe) 

In [38]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.684426,0.684426,0.680243,0.67776,0.515267,0.515267,0.513636,0.512951


**Random Forest**

In [39]:
clf_rf = RandomForestClassifier()

In [40]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier()

In [41]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [42]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    1.0000    1.0000       361
           2     1.0000    1.0000    1.0000       271
           3     1.0000    1.0000    1.0000       588

    accuracy                         1.0000      1220
   macro avg     1.0000    1.0000    1.0000      1220
weighted avg     1.0000    1.0000    1.0000      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5852    0.6561    0.6186       157
           2     0.5312    0.1405    0.2222       121
           3     0.6582    0.8455    0.7402       246

    accuracy                         0.6260       524
   macro avg     0.5916    0.5474    0.5270       524
weighted avg     0.6070    0.6260    0.5842       524



In [43]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.684426,0.684426,0.680243,0.67776,0.515267,0.515267,0.513636,0.512951
1,RandomForest,1.000000,1.000000,1.000000,1.00000,0.625954,0.625954,0.607034,0.584169


**AdaBoost**

In [44]:
clf_adaboost =  AdaBoostClassifier()

In [45]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [46]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [47]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.7324    0.7202    0.7263       361
           2     0.5120    0.3137    0.3890       271
           3     0.6981    0.8299    0.7584       588

    accuracy                         0.6828      1220
   macro avg     0.6475    0.6213    0.6245      1220
weighted avg     0.6669    0.6828    0.6668      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.4938    0.5032    0.4984       157
           2     0.3333    0.2066    0.2551       121
           3     0.5986    0.7033    0.6467       246

    accuracy                         0.5286       524
   macro avg     0.4752    0.4710    0.4668       524
weighted avg     0.5059    0.5286    0.5119       524



In [48]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.684426,0.684426,0.680243,0.677760,0.515267,0.515267,0.513636,0.512951
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.625954,0.625954,0.607034,0.584169
2,Adaboost,0.682787,0.682787,0.666939,0.666814,0.528626,0.528626,0.505938,0.511861


**Gradient Boosting Classifier**

In [49]:
clf_gbm = GradientBoostingClassifier()

In [50]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [51]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [52]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9664    0.9557    0.9610       361
           2     0.9920    0.9114    0.9500       271
           3     0.9430    0.9847    0.9634       588

    accuracy                         0.9598      1220
   macro avg     0.9671    0.9506    0.9581      1220
weighted avg     0.9608    0.9598    0.9597      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5858    0.6306    0.6074       157
           2     0.3148    0.1405    0.1943       121
           3     0.6478    0.7927    0.7130       246

    accuracy                         0.5935       524
   macro avg     0.5162    0.5213    0.5049       524
weighted avg     0.5524    0.5935    0.5616       524



In [53]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.684426,0.684426,0.680243,0.677760,0.515267,0.515267,0.513636,0.512951
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.625954,0.625954,0.607034,0.584169
2,Adaboost,0.682787,0.682787,0.666939,0.666814,0.528626,0.528626,0.505938,0.511861
3,GBM,0.959836,0.959836,0.960796,0.959711,0.593511,0.593511,0.552351,0.561560


**SVM Model**

In [54]:
linear_svm = SVC(kernel='linear', C=1)

In [55]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, kernel='linear')

In [56]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [57]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6898    0.8255    0.7516       361
           2     0.7698    0.3579    0.4887       271
           3     0.7764    0.8741    0.8224       588

    accuracy                         0.7451      1220
   macro avg     0.7454    0.6859    0.6875      1220
weighted avg     0.7493    0.7451    0.7273      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5373    0.6879    0.6034       157
           2     0.2727    0.1240    0.1705       121
           3     0.6418    0.6992    0.6693       246

    accuracy                         0.5630       524
   macro avg     0.4839    0.5037    0.4810       524
weighted avg     0.5253    0.5630    0.5343       524



In [58]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.684426,0.684426,0.680243,0.677760,0.515267,0.515267,0.513636,0.512951
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.625954,0.625954,0.607034,0.584169
2,Adaboost,0.682787,0.682787,0.666939,0.666814,0.528626,0.528626,0.505938,0.511861
3,GBM,0.959836,0.959836,0.960796,0.959711,0.593511,0.593511,0.552351,0.561560
4,SVM,0.745082,0.745082,0.749339,0.727310,0.562977,0.562977,0.525265,0.534331


**KNN**

In [59]:
knn = KNeighborsClassifier(n_neighbors=7)

In [60]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [61]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [62]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6150    0.7036    0.6563       361
           2     0.5395    0.3026    0.3877       271
           3     0.7008    0.7806    0.7385       588

    accuracy                         0.6516      1220
   macro avg     0.6184    0.5956    0.5942      1220
weighted avg     0.6396    0.6516    0.6363      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5183    0.6306    0.5690       157
           2     0.2712    0.1322    0.1778       121
           3     0.6606    0.7358    0.6962       246

    accuracy                         0.5649       524
   macro avg     0.4834    0.4995    0.4810       524
weighted avg     0.5280    0.5649    0.5383       524



In [63]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.684426,0.684426,0.680243,0.677760,0.515267,0.515267,0.513636,0.512951
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.625954,0.625954,0.607034,0.584169
2,Adaboost,0.682787,0.682787,0.666939,0.666814,0.528626,0.528626,0.505938,0.511861
3,GBM,0.959836,0.959836,0.960796,0.959711,0.593511,0.593511,0.552351,0.561560
4,SVM,0.745082,0.745082,0.749339,0.727310,0.562977,0.562977,0.525265,0.534331
5,KNN,0.651639,0.651639,0.639562,0.636281,0.564885,0.564885,0.528042,0.538345


**GNB**

In [64]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [65]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [66]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [67]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3816    0.7950    0.5157       361
           2     0.2459    0.0554    0.0904       271
           3     0.6953    0.4813    0.5688       588

    accuracy                         0.4795      1220
   macro avg     0.4410    0.4439    0.3916      1220
weighted avg     0.5027    0.4795    0.4468      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3655    0.7962    0.5010       157
           2     0.2400    0.0496    0.0822       121
           3     0.6561    0.4187    0.5112       246

    accuracy                         0.4466       524
   macro avg     0.4205    0.4215    0.3648       524
weighted avg     0.4729    0.4466    0.4091       524



In [68]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.684426,0.684426,0.680243,0.677760,0.515267,0.515267,0.513636,0.512951
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.625954,0.625954,0.607034,0.584169
2,Adaboost,0.682787,0.682787,0.666939,0.666814,0.528626,0.528626,0.505938,0.511861
3,GBM,0.959836,0.959836,0.960796,0.959711,0.593511,0.593511,0.552351,0.561560
4,SVM,0.745082,0.745082,0.749339,0.727310,0.562977,0.562977,0.525265,0.534331
5,KNN,0.651639,0.651639,0.639562,0.636281,0.564885,0.564885,0.528042,0.538345
6,GNB,0.479508,0.479508,0.502680,0.446840,0.446565,0.446565,0.472923,0.409064
